In [1]:
from imutils.video import VideoStream
from imutils import face_utils
import argparse
import imutils
import time
import dlib
import cv2
import numpy as np
from keras.models import model_from_json

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

target = ['angry','disgust','fear','happy','sad','surprise','neutral']

def load_model(name):
    model = model_from_json(open(name+'_architecture.json').read())
    model.load_weights(name + '_weights.h5')
    return(model)

In [3]:
detector = dlib.get_frontal_face_detector()
# predictor = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")

predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')

In [4]:
vc = cv2.VideoCapture(0)
currentFrame = 0
time.sleep(2.0)

#load model train
model = load_model("model1")

In [ ]:
while(True):
    ret ,frame = vc.read()
    frame = imutils.resize(frame, width = 400)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray, 0)
#     print(faces)
    if len(faces) > 0:
        text = "{} face(s) found".format(len(faces))
        cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 255), 2)
        
    for face in faces:
        #bounding box
        (bx, by, bH, bW) = face_utils.rect_to_bb(face)
        cv2.rectangle(frame, (bx, by), (bx+bW, by+bH), (0,0,255),1)
         
        face_crop = frame[by:by+bH,bx:bx+bW]
        face_crop = cv2.resize(face_crop,(48,48))
 

        face_crop = (face_crop.astype('float32') - np.mean(face_crop.astype('float32'))) / 256
        face_crop = np.asarray(face_crop)
        
        print(face_crop.shape)
        face_crop = face_crop.reshape(1, face_crop.shape[0],face_crop.shape[1],1)
        
        
        prediction = model.predict_classes(face_crop, batch_size=64)
        
        result = target[prediction[0]]
        cv2.putText(frame,result,(bx,by), cv2.FONT_HERSHEY_SIMPLEX, 1, (200,0,0), 3, cv2.FONT_HERSHEY_SIMPLEX)
        #determine landmark points and revert numpy array
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)
        
        for (i, (x, y)) in enumerate(shape):
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
            cv2.putText(frame, str(i + 1), (x - 10, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
        
    #show the frame
    cv2.imshow("Video", frame)
    key = cv2.waitKey(1) & 0xFF
    
    #if the key == q, break
    if key == ord('q'):
        break

[[[-0.14713089 -0.17056839 -0.15884964]
  [-0.16666214 -0.17838089 -0.16666214]
  [-0.15884964 -0.17447464 -0.15494339]
  ...
  [ 0.19271286  0.13411911  0.11068161]
  [ 0.3059941   0.3294316   0.33333784]
  [ 0.33333784  0.3216191   0.33333784]]

 [[-0.15103714 -0.17056839 -0.16666214]
  [-0.15494339 -0.16275589 -0.17056839]
  [-0.15103714 -0.17447464 -0.15103714]
  ...
  [ 0.02083786 -0.03775589 -0.05728714]
  [ 0.32552534  0.3294316   0.31771284]
  [ 0.3372441   0.3372441   0.34115034]]

 [[-0.15494339 -0.17447464 -0.16275589]
  [-0.15494339 -0.17838089 -0.16275589]
  [-0.15884964 -0.17447464 -0.15494339]
  ...
  [-0.04947464 -0.09244339 -0.11197464]
  [ 0.3216191   0.3216191   0.31771284]
  [ 0.33333784  0.3372441   0.33333784]]

 ...

 [[ 0.4075566   0.4075566   0.3763066 ]
  [ 0.02083786  0.01693161  0.07161911]
  [-0.09634964 -0.13931839 -0.11588089]
  ...
  [ 0.38802534  0.36458784  0.28646284]
  [ 0.3606816   0.3372441   0.2591191 ]
  [ 0.37240034  0.3372441   0.2591191 ]]

 [

[[[ 0.10692003  0.10301378  0.18504503]
  [ 0.11082628  0.11473253  0.20848253]
  [ 0.09910753  0.09910753  0.18895128]
  ...
  [-0.11182997 -0.13136122 -0.11182997]
  [-0.14307997 -0.15479872 -0.14698622]
  [-0.15089247 -0.15479872 -0.15870497]]

 [[ 0.12254503  0.13035753  0.24363878]
  [ 0.13035753  0.13817003  0.23973253]
  [ 0.10692003  0.11473253  0.20067003]
  ...
  [-0.08448622 -0.10792372 -0.08839247]
  [-0.13526747 -0.15089247 -0.14307997]
  [-0.14698622 -0.16261122 -0.15089247]]

 [[ 0.12645128  0.14207628  0.23582628]
  [ 0.10692003  0.13035753  0.23973253]
  [ 0.09520128  0.11863878  0.21238878]
  ...
  [-0.05714247 -0.08448622 -0.03370497]
  [-0.11964247 -0.13917372 -0.11182997]
  [-0.13917372 -0.15089247 -0.13917372]]

 ...

 [[-0.12745497 -0.13526747 -0.08839247]
  [-0.08448622 -0.11182997 -0.08057997]
  [-0.05323622 -0.10011122 -0.06495497]
  ...
  [ 0.42332628  0.39207628  0.33348253]
  [ 0.44676378  0.41942003  0.35692003]
  [ 0.40770128  0.40379503  0.32176378]]

 [

[[[-0.14175868 -0.18472743 -0.15347743]
  [-0.12222743 -0.17300868 -0.13003993]
  [-0.09097743 -0.09878993 -0.04800868]
  ...
  [-0.18863368 -0.20035243 -0.18863368]
  [-0.16519618 -0.18472743 -0.16519618]
  [ 0.04964757  0.02621007  0.01058507]]

 [[-0.16128993 -0.18472743 -0.13785243]
  [-0.13003993 -0.15738368 -0.09878993]
  [-0.02457118 -0.04019618  0.04574132]
  ...
  [-0.17691493 -0.19644618 -0.18472743]
  [-0.16128993 -0.18863368 -0.17300868]
  [ 0.01839757 -0.01675868 -0.03238368]]

 [[-0.11832118 -0.16519618 -0.11832118]
  [-0.06753993 -0.09488368 -0.03628993]
  [ 0.03402257  0.03792882  0.11605382]
  ...
  [-0.17691493 -0.19253993 -0.19644618]
  [-0.16128993 -0.18472743 -0.17691493]
  [-0.02066493 -0.04800868 -0.06753993]]

 ...

 [[ 0.46761632  0.43246007  0.38167882]
  [ 0.46761632  0.44808507  0.40121007]
  [ 0.35824132  0.35042882  0.32699132]
  ...
  [ 0.40511632  0.37777257  0.31917882]
  [ 0.34652257  0.31527257  0.24105382]
  [ 0.37386632  0.35824132  0.28792882]]

 [

[[[-0.13973151 -0.12801276 -0.10848151]
  [-0.08113776 -0.06160651 -0.01082526]
  [-0.00301276  0.03604974  0.11417474]
  ...
  [-0.15535651 -0.17098151 -0.15926276]
  [-0.16316901 -0.16707526 -0.17098151]
  [-0.15145026 -0.16707526 -0.15535651]]

 [[-0.09285651 -0.08895026 -0.03426276]
  [ 0.00870599  0.03604974  0.11417474]
  [ 0.05167474  0.09073724  0.18448724]
  ...
  [-0.15535651 -0.17098151 -0.15535651]
  [-0.15926276 -0.16707526 -0.16316901]
  [-0.14363776 -0.15926276 -0.14363776]]

 [[-0.05770026 -0.03426276  0.02823724]
  [ 0.05558099  0.09073724  0.18839349]
  [ 0.09854974  0.12979974  0.24698724]
  ...
  [-0.15535651 -0.15535651 -0.15926276]
  [-0.15535651 -0.17098151 -0.15926276]
  [-0.13582526 -0.14754401 -0.13582526]]

 ...

 [[ 0.555581    0.539956    0.48526847]
  [ 0.5165185   0.524331    0.49308097]
  [ 0.39542472  0.39151847  0.36026847]
  ...
  [-0.09285651 -0.11629401 -0.08113776]
  [-0.08504401 -0.09676276 -0.06160651]
  [-0.07332526 -0.06160651 -0.04598151]]

 [

[[[ 4.69041049e-01  5.04197299e-01  5.00291049e-01]
  [ 4.19729948e-03 -4.65839505e-02 -5.04902005e-02]
  [-1.44240201e-01 -1.55958951e-01 -1.44240201e-01]
  ...
  [-2.02833951e-01 -2.18458951e-01 -2.06740201e-01]
  [-2.02833951e-01 -2.14552701e-01 -2.02833951e-01]
  [-2.06740201e-01 -2.18458951e-01 -2.06740201e-01]]

 [[ 3.71384799e-01  3.94822299e-01  3.83103549e-01]
  [-8.56464505e-02 -1.16896451e-01 -1.05177701e-01]
  [-1.40333951e-01 -1.63771451e-01 -1.36427701e-01]
  ...
  [-1.91115201e-01 -1.98927701e-01 -1.91115201e-01]
  [-2.02833951e-01 -2.14552701e-01 -2.02833951e-01]
  [-2.18458951e-01 -2.10646451e-01 -2.10646451e-01]]

 [[ 2.38572299e-01  2.73728549e-01  2.65916049e-01]
  [-1.20802701e-01 -1.24708951e-01 -1.09083951e-01]
  [-1.48146451e-01 -1.71583951e-01 -1.48146451e-01]
  ...
  [-1.55958951e-01 -1.83302701e-01 -1.55958951e-01]
  [-2.14552701e-01 -2.14552701e-01 -2.18458951e-01]
  [-2.06740201e-01 -2.18458951e-01 -2.06740201e-01]]

 ...

 [[ 4.65134799e-01  4.65134799e-01

...

[[[-0.01722266 -0.04066016  0.08043359]
  [ 0.05699609  0.06871484  0.20933984]
  [ 0.12340234  0.13902734  0.2835586 ]
  ...
  [-0.12659766 -0.14222266 -0.07972266]
  [-0.16956641 -0.17347266 -0.12269141]
  [-0.16956641 -0.18519141 -0.12269141]]

 [[ 0.02574609  0.02183984  0.08433984]
  [ 0.10777734  0.11949609  0.20543359]
  [ 0.13902734  0.16246484  0.27965236]
  ...
  [-0.09534766 -0.13441016 -0.10706641]
  [-0.15003516 -0.16566016 -0.14222266]
  [-0.16566016 -0.17737891 -0.16956641]]

 [[ 0.04527734  0.02965234  0.10387109]
  [ 0.14683984  0.15855859  0.2601211 ]
  [ 0.15855859  0.17027734  0.30308986]
  ...
  [-0.06019141 -0.10316016 -0.06019141]
  [-0.13441016 -0.15784766 -0.12659766]
  [-0.16175391 -0.18128516 -0.16956641]]

 ...

 [[-0.12269141 -0.14612891 -0.09925391]
  [-0.10316016 -0.15003516 -0.11487891]
  [ 0.11558984  0.06480859  0.08433984]
  ...
  [ 0.3616836   0.34996486  0.2757461 ]
  [ 0.3226211   0.3148086   0.24449609]
  [ 0.3694961   0.34215236  0.27183986]]

 [

[[[-0.06303066 -0.07865566  0.01509434]
  [ 0.02290684  0.03853184  0.17134434]
  [ 0.08931309  0.11275059  0.2611881 ]
  ...
  [-0.10209316 -0.13334316 -0.04349941]
  [-0.15287441 -0.17240566 -0.10599941]
  [-0.18412441 -0.19193691 -0.13724941]]

 [[-0.03959316 -0.07084316 -0.02396816]
  [ 0.08150059  0.09712559  0.17915684]
  [ 0.12837559  0.15181309  0.22993809]
  ...
  [-0.09037441 -0.12162441 -0.07474941]
  [-0.14115566 -0.16849941 -0.12943691]
  [-0.16068691 -0.18412441 -0.16849941]]

 [[-0.07084316 -0.07865566 -0.01615566]
  [ 0.09321934  0.12446934  0.21431309]
  [ 0.13228184  0.16353184  0.26509434]
  ...
  [-0.07865566 -0.10599941 -0.05521816]
  [-0.14115566 -0.14506191 -0.11381191]
  [-0.15678066 -0.17631191 -0.16459316]]

 ...

 [[-0.11771816 -0.16068691 -0.12162441]
  [-0.09428066 -0.12943691 -0.09428066]
  [ 0.04243809  0.05415684  0.06587559]
  ...
  [ 0.3627506   0.35103184  0.30415684]
  [ 0.3471256   0.32759434  0.28071934]
  [ 0.36665684  0.3393131   0.2768131 ]]

 [

[[[-0.02401847 -0.07479972  0.04629403]
  [ 0.05801278  0.03848153  0.18691903]
  [ 0.08145028  0.08926278  0.24551278]
  ...
  [-0.08651847 -0.11776847 -0.03183097]
  [-0.11776847 -0.16073722 -0.09042472]
  [-0.16854972 -0.18026847 -0.11776847]]

 [[ 0.00723153 -0.02401847  0.03457528]
  [ 0.08926278  0.09707528  0.18691903]
  [ 0.12441903  0.14004403  0.22988778]
  ...
  [-0.05917472 -0.10214347 -0.05526847]
  [-0.10995597 -0.14901847 -0.12167472]
  [-0.16073722 -0.17245597 -0.16073722]]

 [[-0.05917472 -0.08261222 -0.01229972]
  [ 0.10488778  0.11660653  0.21816903]
  [ 0.13223153  0.15176278  0.28066903]
  ...
  [-0.04745597 -0.08261222 -0.03183097]
  [-0.10604972 -0.14120597 -0.08261222]
  [-0.16073722 -0.17636222 -0.15292472]]

 ...

 [[-0.11386222 -0.15292472 -0.12558097]
  [-0.03573722 -0.07479972 -0.05526847]
  [ 0.09316903  0.06191903  0.10488778]
  ...
  [ 0.35488778  0.35488778  0.31191903]
  [ 0.35098153  0.33535653  0.28848153]
  [ 0.37832528  0.34707528  0.27676278]]

 [

[[[-0.08343619 -0.09515494  0.00250131]
  [-0.01312369  0.00640756  0.13922006]
  [ 0.06500131  0.06500131  0.20953256]
  ...
  [-0.07562369 -0.13031119 -0.01702994]
  [-0.12640494 -0.15765494 -0.07952994]
  [-0.16937369 -0.18499869 -0.13031119]]

 [[-0.07952994 -0.08734244 -0.04437369]
  [ 0.04547006  0.06109506  0.15093881]
  [ 0.07281381  0.09625131  0.19390756]
  ...
  [-0.04437369 -0.09515494 -0.04437369]
  [-0.10296744 -0.14593619 -0.11077994]
  [-0.15374869 -0.17718619 -0.16546744]]

 [[-0.09906119 -0.09906119 -0.04437369]
  [ 0.04156381  0.09234506  0.16656381]
  [ 0.08062631  0.11578256  0.19781381]
  ...
  [-0.04827994 -0.07171744 -0.02093619]
  [-0.09124869 -0.12249869 -0.07562369]
  [-0.14593619 -0.17327994 -0.14593619]]

 ...

 [[-0.13031119 -0.16546744 -0.13421744]
  [-0.04827994 -0.09906119 -0.07171744]
  [ 0.09625131  0.05328256  0.08843881]
  ...
  [ 0.36578256  0.35015756  0.30328256]
  [ 0.35015756  0.33453256  0.27984506]
  [ 0.3775013   0.3462513   0.2837513 ]]

 [

[[[-0.13738619 -0.14910494 -0.13738619]
  [-0.11785494 -0.14910494 -0.13347994]
  [-0.11004244 -0.13347994 -0.11785494]
  ...
  [-0.16472994 -0.18035494 -0.16863619]
  [-0.16082369 -0.17644869 -0.16472994]
  [-0.16472994 -0.18035494 -0.16863619]]

 [[-0.14519869 -0.15301119 -0.13347994]
  [-0.14129244 -0.14129244 -0.10222994]
  [-0.09051119 -0.09832369 -0.03582369]
  ...
  [-0.17254244 -0.17644869 -0.17254244]
  [-0.16863619 -0.18035494 -0.16863619]
  [-0.16082369 -0.16863619 -0.16082369]]

 [[-0.10613619 -0.12957369 -0.09051119]
  [-0.08660494 -0.09832369 -0.04754244]
  [-0.01238619 -0.00847994  0.07355131]
  ...
  [-0.15691744 -0.17254244 -0.16082369]
  [-0.16863619 -0.18426119 -0.17254244]
  [-0.16082369 -0.17254244 -0.16472994]]

 ...

 [[-0.07097994 -0.11785494 -0.05535494]
  [-0.11004244 -0.14519869 -0.10613619]
  [-0.14129244 -0.18035494 -0.13738619]
  ...
  [ 0.3860513   0.3626138   0.28839505]
  [ 0.4251138   0.41339505  0.31964505]
  [ 0.38214505  0.38214505  0.33527005]]

 [

In [ ]:
#do a bit of cleanup
cv2.destroyAllwindows()
vc.stop()
